# scraping data for doc2vec


Because the data which is used train doc2vec model needs string data, so I'm going to scrape more string information from coursera websites.

## Connect the mongoDB to get the urls

In [1]:
import pymongo
from pymongo import MongoClient
from pprint import pprint
import codecs

In [2]:
client=MongoClient('mongodb://localhost:27017')

In [3]:
client.list_database_names()

['admin', 'config', 'coursera', 'enron', 'local']

In [4]:
db=client.coursera

In [5]:
db.list_collection_names()

['courseinfo_v2']

In [6]:
collection=db.courseinfo_v2

In [7]:
collection.count_documents({})

2189

In [8]:
for i in collection.find({}).limit(5):
    pprint(i)

{'_id': ObjectId('5c27d541efb65d376b07721c'),
 'category': {'1': 'Arts and Humanities', '2': 'Music and Art'},
 'chapters': ['音乐的史纲与音乐的风格',
              '音乐的空间与音乐的时间',
              '20世纪西方音乐的多元新秩序特征',
              '世纪末情结下的法国音乐：德彪西与拉威尔'],
 'cid': '20cnwm',
 'cname': '20世纪西方音乐 Western Music in the 20th Century',
 'faid': True,
 'hours': '17',
 'instrlist': {'毕明辉 Bryan Minghui BI': ['20世纪西方音乐 Western Music in the 20th '
                                        'Century']},
 'language': 'Chinese (Simplified)',
 'num_rates': '23',
 'num_reviews': '4',
 'rates': '4.3',
 'skills': [],
 'subtitles': 'Subtitles: Chinese (Simplified)',
 'uniname': ['Peking University']}
{'_id': ObjectId('5c27d541efb65d376b07721d'),
 'category': {'1': 'Business', '2': 'Business Essentials'},
 'chapters': ['Introduction',
              'IT and the CIO in the Digital World',
              'Steer the Balance Sheet',
              'Market and Sell Products'],
 'cid': '2-speed-it',
 'cname': 'Two Speed IT: How Compa

**unset some fields, which can be used to get the contents about the certain courses**

In [35]:
# use this can update the content of the documents to unset some fields.
db.collection_enlarged.update_many({},{"$unset":{"faid":""}})

In [77]:
n=0
for i in db.collection_enlarged.find({}).limit(2):
    db.collection_enlarged.update_one({"_id":i["_id"]},{
        "$set":{"test":n}
    })
    pprint(i)
    print(type(i)) # dict

{'_id': ObjectId('5c27d541efb65d376b07721d'),
 'category': {'1': 'Business', '2': 'Business Essentials'},
 'chapters': ['Introduction',
              'IT and the CIO in the Digital World',
              'Steer the Balance Sheet',
              'Market and Sell Products'],
 'cid': '2-speed-it',
 'cname': 'Two Speed IT: How Companies Can Surf the Digital Wave, a BCG '
          'Perspective',
 'hours': '22',
 'instrlist': {'Antoine Gourévitch': ['Two Speed IT: How Companies Can Surf '
                                      'the Digital Wave, a BCG Perspective'],
               'Eric Baudson': ['Two Speed IT: How Companies Can Surf the '
                                'Digital Wave, a BCG Perspective'],
               'Vanessa Lyon': ['Two Speed IT: How Companies Can Surf the '
                                'Digital Wave, a BCG Perspective']},
 'language': 'English',
 'num_rates': '128',
 'num_reviews': '33',
 'rates': '4.4',
 'skills': [],
 'subtitles': 'Subtitles: English',
 'uniname'

In [21]:
# here I get the url list for the new scrapying work
domain="https://www.coursera.org/learn/"
for i in collection.find({}):
    codecs.open("url_list_enlarged.txt","a+",'utf-8').write(i['cname']+"\t"+domain+i['cid']+"\n")

## Scrapy code

In [9]:
import codecs
import re
from pprint import pprint
from urllib.request import urlopen
from urllib.error import HTTPError
from bs4 import BeautifulSoup
import json

In [10]:
def open_soup(url):
    try:
        html=urlopen(url).read().decode('utf-8')
        return BeautifulSoup(html, features='lxml')
    except HTTPError:
        print("\nThe site has not found(404):\t",url)
        return False 
def get_description(soup):
    return soup.find("div",class_="content-inner").contents[0]

def get_contents(soup):
    contents=soup.find_all("div",class_="P_gjs17i-o_O-weightNormal_s9jwp5-o_O-fontBody_56f0wi")
    contentlist=[]
    for i in contents:
        try:
            c=i.contents[0].contents[0].contents[1].contents[0]
            contentlist.append(c)
        except:
            continue
    return contentlist

the features of the new data:
1. course name
2. category(can be the label)


3. description (don't have this one, only need to get this one)
4. chapters and its contents


5. skills (it is already the list of words)

**apart from the description, the information are fetched from the mongoDB**

It maybe faster.

Addd new field: **description**, **chapter contents**

In [12]:
domain="https://www.coursera.org/learn/"
# for i in collection.find({}):
#     url=domain+i['cid']
#     soup=open_soup(url)
#     if not soup:
#         continue
#     des=get_description(soup)
#     con=get_contents(soup)        
#     collection.update_one({"_id":i["_id"]},{"$set":{"description":des,"contents":con}})
#     print(i)
#     print(i['cid'],"successful...")


In [85]:
# soup=open_soup("https://www.coursera.org/learn/machine-learning")
des=get_description(soup)
print(des)

Machine learning is the science of getting computers to act without being explicitly programmed. In the past decade, machine learning has given us self-driving cars, practical speech recognition, effective web search, and a vastly improved understanding of the human genome. Machine learning is so pervasive today that you probably use it dozens of times a day without knowing it. Many researchers also think it is the best way to make progress towards human-level AI. In this class, you will learn about the most effective machine learning techniques, and gain practice implementing them and getting them to work for yourself. More importantly, you'll learn about not only the theoretical underpinnings of learning, but also gain the practical know-how needed to quickly and powerfully apply these techniques to new problems. Finally, you'll learn about some of Silicon Valley's best practices in innovation as it pertains to machine learning and AI.

This course provides a broad introduction to ma